### Basic Setup

#### Authentication

In [1]:
import requests
from requests_oauthlib import OAuth1
import urllib
auth = OAuth1('uFgL5vbME7Gi8Zv6uCO3g',
           'PN9g4sbyTh67n2WQmYJvMPrFwbyeKG9UjCjiA2qlnGI',
           '884076750-EuiDLe8HzbPNk5G9aI1XE12qh8Bs1pIsgHi02ipx',
           '1yWYyLmsHTrenvvYGqZOq26lT0vzWWsbgk6T3cknx3Q')

#### URL Encoding

In [2]:
searchQuery = urllib.urlencode({'q':'Donald Trump','count':100})

print searchQuery

q=Donald+Trump&count=100


### Functions to be used for scraping

In [3]:
def getRow(status, row ):
    row = getBasicInfo(status, row)
    row = getEntities(status, row)
    row = getUser(status, row)
    row = getOriginalStats(status, row)
    row = getOriginalUser(status, row)
    return row;

In [4]:
#Functions related to basic information

def getBasicInfo( status , row ):
    created_at = status['created_at']
    tweet_id = status['id_str']
    retweet_count = status['retweet_count']
    reply_user_id = status['in_reply_to_user_id_str']
    reply_status_id = status['in_reply_to_status_id_str']
    text = status['text']
    
    row.append(created_at)
    row.append(tweet_id)
    row.append(retweet_count)
    row.append(reply_user_id)
    row.append(reply_status_id)
    row.append(text)
    
    return row

In [5]:
#Functions related to entities

def getEntities( status , row):
    tags = getHashTags(status['entities'])
    ids, names = getMentionedUser(status['entities'])
    row.append(tags)
    row.append(ids)
    row.append(names)
    return row

def getHashTags( entity ):
    count = len(entity['hashtags'])
    tags = []
    for i in range(0, count):
        tags.append(entity['hashtags'][i]['text'])
    return tags 


def getMentionedUser( entity ):
    count = len(entity['user_mentions'])
    ids = []
    names = []
    for i in range(0, count):
        ids.append(entity['user_mentions'][i]['id_str'])
        names.append(entity['user_mentions'][i]['name'])
    return ids, names 


In [6]:
#Functions related to user

def getUser( status , row ):
    user = status['user'];
    user_id = user['id_str']
    name = user['name']
    screen_name = user['screen_name']
    friends_count = user['friends_count']
    followers_count = user['followers_count']
    statuses_count = user['statuses_count']
    
    row.append(user_id)
    row.append(name)
    row.append(screen_name)
    row.append(friends_count)
    row.append(followers_count)
    row.append(statuses_count)
    
    return row

In [7]:
#Functions related to original tweet (if current tweet is retweet)

def getOriginalStats( status , row):
    ori_created_at = ''
    ori_tweet_id = ''
    ori_retweet_count = ''
    if 'retweeted_status' in status:
        retweet_status = status['retweeted_status']
        ori_created_at = retweet_status['created_at']
        ori_tweet_id = retweet_status['id_str']
        ori_retweet_count = retweet_status['retweet_count']
    row.append(ori_created_at)
    row.append(ori_tweet_id)
    row.append(ori_retweet_count)
    return row

def getOriginalUser( status , row):
    ori_user_id = ''
    ori_name = ''
    ori_screen_name = ''
    ori_friends_count = ''
    ori_followers_count = ''
    ori_statuses_count = ''
    if 'retweeted_status' in status:
        retweet_user = status['retweeted_status']['user']
        ori_user_id = retweet_user['id_str']
        ori_name = retweet_user['name']
        ori_screen_name = retweet_user['screen_name']
        ori_friends_count = retweet_user['friends_count']
        ori_followers_count = retweet_user['followers_count']
        ori_statuses_count = retweet_user['statuses_count']
    
    row.append(ori_user_id)
    row.append(ori_name)
    row.append(ori_screen_name)
    row.append(ori_friends_count) 
    row.append(ori_followers_count)
    row.append(ori_statuses_count)
    return row


### Scraping

#### Set encoding to prevent error

In [8]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

#### Import Libraries

In [9]:
# import libraries
import json
import unicodecsv as csv
import time
from random import randint
import datetime
import codecs

#### Set Search Query

In [10]:
query = 'Donald Trump'

searchQuery = urllib.urlencode({'q':query,'count':100})
fileQuery = query.replace(' ','')

In [11]:
startTime = datetime.datetime.strftime(datetime.datetime.now(), '%Y_%m_%d-%H_%M_%S')
fileName = "./Data/%s(%s).csv" % (fileQuery, startTime)
indexFileName = "./Data/%s(%s)Query.csv" % (fileQuery, startTime)

In [12]:
labels = ['created_at', 'tweet_id', 'retweet_count', 'reply_user_id', 'reply_status_id', 'text', 'tags', 'ids', 'names',
    'user_id', 'name', 'screen_name', 'friends_count', 'followers_count', 'statuses_count',   
    'ori_created_at', 'ori_tweet_id', 'ori_retweet_count', 'ori_user_id', 
    'ori_name', 'ori_screen_name', 'ori_friends_count', 'ori_followers_count', 'ori_statuses_count']

In [ ]:
url = 'https://api.twitter.com/1.1/search/tweets.json?' + searchQuery
response = requests.get(url, auth=auth)


# Write the label
with codecs.open(fileName, 'ab','utf-8') as fp:
    a = csv.writer(fp, delimiter=',')
    a.writerow(labels)

nTrial = 10000
subTrial = 0;
for trial in range(1,nTrial+1):
    print "At trial", trial
    
    
    data = response.json()
    
    # Save the data to a list
    try : 
        rows=[]
        for i in range(1,len(data['statuses'])):
            tweet = data['statuses'][i]
            row = []
            row = getRow(tweet, row)
            rows.append(row)
    except : 
        if subTrial == 3 : 
            break;
        else : 
            print 'Error raised at List Creating Stage, subTrial %d' %subTrial
            subTrial += 1
            time.sleep(20)
            continue;
            
    try : 
        with codecs.open(fileName, 'ab','utf-8') as fp:
            a = csv.writer(fp, delimiter=',')
            for row in rows:
                a.writerow(row)
    except : 
        if subTrial == 3 : 
            break;
        else : 
            print 'Error raised at Data Writing Stage, subTrial %d' %subTrial
            subTrial += 1
            time.sleep(20)
            continue;
            
    last_Index = len(data['statuses'])-1
    last_minId = str(data['statuses'][last_Index]['id_str'])
    idQuery = urllib.urlencode({'max_id':last_minId})       
            
    #Note how far we have searched
    with open(indexFileName, "ab") as q:
        t = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
        q.write(t + ' : ' +  'Searched up to ' + last_minId + '\n')
        
    subTrial = 0        
    time.sleep(randint(61,70))
    response = requests.get('https://api.twitter.com/1.1/search/tweets.json?'+searchQuery+'&'+idQuery, auth=auth)

####  Starting again after lost connection

In [10]:
query = 'Donald Trump'
searchQuery = urllib.urlencode({'q':query,'count':100})

fileName = './Data/DonaldTrump(2016_01_30-12_45_19).csv'
indexFileName = './Data/DonaldTrump(2016_01_30-12_45_19)Query.csv'
startTrial = 5150
last_minId =  '692595693689663488'
idQuery = urllib.urlencode({'max_id':last_minId})       


In [ ]:
response = requests.get('https://api.twitter.com/1.1/search/tweets.json?'+searchQuery+'&'+idQuery, auth=auth)

nTrial = 10000
subTrial = 0;
for trial in range(startTrial,nTrial+1):
    print "At trial %d for query %s" % (trial, searchQuery)
    
    data = response.json()
    
    # Save the data to a list
    try : 
        rows=[]
        for i in range(1,len(data['statuses'])):
            tweet = data['statuses'][i]
            row = []
            row = getRow(tweet, row)
            rows.append(row)
    except : 
        if subTrial == 3 : 
            break;
        else : 
            print 'Error raised at List Creating Stage, subTrial %d' %subTrial
            subTrial += 1
            time.sleep(20)
            continue;
            
    try : 
        with codecs.open(fileName, 'ab','utf-8') as fp:
            a = csv.writer(fp, delimiter=',')
            for row in rows:
                a.writerow(row)
    except : 
        if subTrial == 3 : 
            break;
        else : 
            print 'Error raised at Data Writing Stage, subTrial %d' %subTrial
            subTrial += 1
            time.sleep(20)
            continue;
            
    last_Index = len(data['statuses'])-1
    last_minId = str(data['statuses'][last_Index]['id_str'])
    idQuery = urllib.urlencode({'max_id':last_minId})       
            
    #Note how far we have searched
    with open(indexFileName, "ab") as q:
        t = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
        q.write(t + ' : ' +  'Searched up to ' + last_minId + '\n')
        
    subTrial = 0        
    time.sleep(randint(61,70))
    response = requests.get('https://api.twitter.com/1.1/search/tweets.json?'+searchQuery+'&'+idQuery, auth=auth)

In [78]:
len(data['statuses'])

99

In [52]:
data['statuses'][99]

{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Sat Jan 30 02:55:28 +0000 2016',
 u'entities': {u'hashtags': [],
  u'symbols': [],
  u'urls': [{u'display_url': u'nationalreview.com/article/430468\u2026',
    u'expanded_url': u'http://www.nationalreview.com/article/430468/donald-trump-what-if-he-wins?utm_campaign=trueAnthem%3A+Trending+Content&utm_content=56ab862d04d3015013dd7c6f&utm_medium=trueAnthem&utm_source=twitter',
    u'indices': [125, 140],
    u'url': u'https://t.co/HnYTqoB0va'}],
  u'user_mentions': [{u'id': 17561562,
    u'id_str': u'17561562',
    u'indices': [3, 9],
    u'name': u'slone',
    u'screen_name': u'slone'},
   {u'id': 25073877,
    u'id_str': u'25073877',
    u'indices': [97, 113],
    u'name': u'Donald J. Trump',
    u'screen_name': u'realDonaldTrump'}]},
 u'favorite_count': 0,
 u'favorited': False,
 u'geo': None,
 u'id': 693266250265337857L,
 u'id_str': u'693266250265337857',
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,

In [16]:
with codecs.open(fileName, 'rb') as csvfileRead:
    reader = csv.reader(csvfileRead, delimiter=',')
    i = 0
    for row in reader:
        if i> 100000 : 
            print row[0]
            break
        i = i +1